In [2]:
using JuMP, Gurobi
import GLPK
import Random
import Plots

In [3]:
function generate_distance_matrix(n; random_seed = 1)
    rng = Random.MersenneTwister(random_seed)
    X = 100 * rand(rng, n)
    Y = 100 * rand(rng, n)
    d = [sqrt((X[i] - X[j])^2 + (Y[i] - Y[j])^2) for i in 1:n, j in 1:n]
    return X, Y, d
end

n = 4
X, Y, d = generate_distance_matrix(n)

([23.603334566204694, 34.651701419196044, 31.27069683360675, 0.790928339056074], [48.86128300795012, 21.096820215853597, 95.1916339835734, 99.99046588986135], [0.0 29.881964531335456 46.960513899062235 55.987491639154314; 29.881964531335456 0.0 74.17191260363222 85.85312621751531; 46.960513899062235 74.17191260363222 0.0 30.855227679377304; 55.987491639154314 85.85312621751531 30.855227679377304 0.0])

In [13]:
d = [
    0 10 15 20;
    10 0 25 25;
    15 25 0 30;
    20 25 30 0;
]


4×4 Matrix{Int64}:
  0  10  15  20
 10   0  35  25
 15  35   0  30
 20  25  30   0

In [335]:
function tsp_dynamic_programming_memoization(distance_matrix)
    n = size(distance_matrix, 1)
    num_states = 2^n
    memo = Dict{Tuple{Int, Int}, Int}()

    function tsp(mask, pos)
        if haskey(memo, (mask, pos))
            return memo[(mask, pos)]
        end

        if mask == 2^n - 1
            return distance_matrix[pos, 1]
        end

        min_cost = Inf
        for next_city in 2:n
            if (mask & (1 << (next_city - 1))) == 0
                new_mask = mask | (1 << (next_city - 1))
                new_cost = distance_matrix[pos, next_city] + tsp(new_mask, next_city)
                min_cost = min(min_cost, new_cost)
            end
        end

        memo[(mask, pos)] = min_cost
        return min_cost
    end

    return tsp(1, 1)
end

# Example usage
distances = [
    0  10 15 20;
    10 0  35 25;
    15 35 0  30;
    20 25 30 0
]

result = tsp_dynamic_programming_memoization(distances)
println("Minimum tour length using memoization: $result")


Minimum tour length using memoization: 80.0


In [346]:
#=
s: starting point 
mask: 지나갔는지 안지나갔는지 확인 
=#

# distance matrix
d =[
    0  10 15 20;
    10 0  35 25;
    15 35 0  30;
    20 25 30 0
]

# define n
rows , cols = size(d)
n = rows

# memo
memo = fill(Inf, (n, 1 << n))

function are_two_set_bits(mask)
    count_set_bits = 0
    while mask > 0
        count_set_bits += mask & 1
        mask >>= 1
    end
    return count_set_bits == 2
end

function indices_of_set_bits(mask)
    indices = Int[]
    position = 1
    
    while mask > 0
        if mask & 1 == 1
            push!(indices, position)
        end
        mask >>= 1
        position += 1
    end
    return indices
end

function tsp_dynamic_programming(mask, s)
    # s는 지나감
    mask = mask | (1 << (s-1))
    
    # base case, 먼가 여기서 잘못된거 같은데... base case어떻게 고쳐야 할까요? -> 맞게 고친것 같은데 
    indices = indices_of_set_bits(mask)
    if are_two_set_bits(mask)
        memo[s, mask] = 0
    end

    #memoization
    if memo[s, mask] != Inf
        return memo[s, mask]
    end
    res = Inf
    for i in 1:n
        if mask & (1 << (i-1)) == 0 && i != s
            updated_mask = mask | (1 << (i-1))
            val = tsp_dynamic_programming(updated_mask, i) + d[i, s]
            res = val # res 값이 업데이트가 되지 않는다. 
        end
    end
    memo[s, mask] = res
    
    return res
end

ans = Inf
for j in 1:n
    ans = min(ans, tsp_dynamic_programming((1 << (n-1)) - 1, j) + d[1, j])
end
println("The cost of most efficient tour = $ans")

The cost of most efficient tour = Inf


In [306]:
function generate_distance_matrix(n; random_seed = 1)
    rng = Random.MersenneTwister(random_seed)
    X = 100 * rand(rng, n)
    Y = 100 * rand(rng, n)
    d = [sqrt((X[i] - X[j])^2 + (Y[i] - Y[j])^2) for i in 1:n, j in 1:n]
    return X, Y, d
end

n = 20
X, Y, d = generate_distance_matrix(n)

([23.603334566204694, 34.651701419196044, 31.27069683360675, 0.790928339056074, 48.86128300795012, 21.096820215853597, 95.1916339835734, 99.99046588986135, 25.166218303197184, 98.66663668987997, 55.57510873245724, 43.71079746096251, 42.471785049513144, 77.3223048457377, 28.11902322857298, 20.947237319807076, 25.137920979222493, 2.037486871266725, 28.77015122756894, 85.9512136087661], [7.695088688120899, 64.03962459899388, 87.35441302706855, 27.85824200287785, 75.13126327861701, 64.4883353942093, 7.782644396003469, 84.81854810000327, 8.56351682044918, 55.32055454580578, 46.33502459235987, 18.582130997265377, 11.198087695816717, 97.6311881619359, 5.161462067432709, 53.80295812064833, 45.56920516275036, 27.93951106725605, 17.824610354168602, 54.89828719625274], [0.0 57.417533359853884 … 11.371156651593584 78.20102282262022; 57.417533359853884 0.0 … 46.587768505328015 52.107619408284236; … ; 11.371156651593584 46.587768505328015 … 0.0 68.1478643072081; 78.20102282262022 52.107619408284236 

In [326]:
function tsp_dynamic_programming()
    memo = fill(Inf, n, 1 << n)
    memo[1, 1] = 0
    
    for mask in 1:(1 << n) - 1
        for s in 1:n
            if (mask & (1 << (s - 1))) != 0
                for j in 1:n
                    if (mask & (1 << (j - 1))) == 0
                        memo[j, mask | (1 << (j - 1))] = min(memo[j, mask | (1 << (j - 1))], memo[s, mask] + d[s, j])
                    end
                end
            end
        end
    end
    
    ans = 10^9
    for i in 2:n
        ans = min(ans, memo[i, (1 << n) - 1] + d[i, 1])
    end
    
    return ans
end

result = tsp_dynamic_programming()
println("The cost of the most efficient tour = $result")


The cost of the most efficient tour = 80.0


In [ ]:
i = 5 # Replace this with your desired value of i
result = (1 << (i-1)) -1

println("Result in binary: ", string(result, base=2))

In [312]:
function indices_of_set_bits(mask)
    indices = Int[]
    position = 1
    
    while mask > 0
        if mask & 1 == 1
            push!(indices, position)
        end
        mask >>= 1
        position += 1
    end
    
    return indices
end

indices_of_set_bits (generic function with 1 method)

In [316]:
mask = 0b01010
print(indices_of_set_bits(mask))

[2, 4]

In [323]:
function tsp_dynamic_programming(mask, s)
    println("Entering tsp_dynamic_programming with mask=$mask, s=$s")

    # visited[s] = 1, s는 지나감
    mask = mask | (1 << (s-1))
    
    # base case
    indices = indices_of_set_bits(mask)
    if are_two_set_bits(mask)
        return d[indices[1], indices[2]]
    end

    # memoization
    if memo[s, mask] != Inf
        return memo[s, mask]
    end

    res = 10^9
    for i in 1:n
        if mask & (1 << (i-1)) == 0 && i != s 
            updated_mask = mask | (1 << (i-1))
            res = min(res, tsp_dynamic_programming(updated_mask, i) + d[i, s])
        end
    end

    memo[s, mask] = res
    println("Exiting tsp_dynamic_programming with mask=$mask, s=$s, res=$res")
    return res
end


tsp_dynamic_programming (generic function with 3 methods)